In [1]:
using Geant4
using Geant4.SystemOfUnits
#using GLMakie, Rotations, IGLWrap_jll  # to force loding G4Vis extension
using CairoMakie
using GeometryBasics, Rotations, IGLWrap_jll

In [2]:
mutable struct CrstDetector <: G4JLDetector
    # main input parameters
    const checkOverlaps::Bool
    const crstXY::Float64
    const crstZ::Float64
    const crstName::String
    
    fCrstMaterial::CxxPtr{G4Material}
    fWrldMaterial::CxxPtr{G4Material}
    
    function CrstDetector(;  crstXY::Float64=3.5mm, 
                             crstZ::Float64=20.0mm,
                             checkOverlaps::Bool=true,
                             crstName="CsI")
        self =  new(checkOverlaps, crstXY, crstZ, crstName)
        nist     = G4NistManager!Instance()
        
        self.fWrldMaterial  = FindOrBuildMaterial(nist, "G4_AIR")
        if crstName == "LYSO"
            cm3      = cm * cm * cm
            self.fCrstMaterial = G4Material("lyso", density= 7.4*g/cm3, ncomponents=4)
            
            Lu = FindOrBuildElement(nist,"Lu")
            Si = FindOrBuildElement(nist,"Si")
            O  = FindOrBuildElement(nist,"O")
            Y  = FindOrBuildElement(nist,"Y")
            AddElement(lyso, Lu, fractionmass=.71)
            AddElement(lyso, Si, fractionmass=.07)
            AddElement(lyso, O, fractionmass=.18)
            AddElement(lyso, Y, fractionmass=.04)
        elseif crstName == "BGO"
            self.fCrstMaterial  = FindOrBuildMaterial(nist, "G4_BGO")
        else
            self.fCrstMaterial  = FindOrBuildMaterial(nist, "G4_CESIUM_IODIDE")
        end
            
       return self 
    end
end

In [3]:
function crstDetectorConstruction(det::CrstDetector)::CxxPtr{G4VPhysicalVolume}
    (; checkOverlaps, crstXY, crstZ)  = det
    crystBox = G4Box("CBox", crstXY/2,crstXY/2,crstZ/2)

    world_sizeXY = 2* crstXY
    world_sizeZ  = 2* crstZ
    
    solidworld = G4Box("World",  0.5*world_sizeXY, 0.5*world_sizeXY, 0.5*world_sizeZ)   
    logicworld = G4LogicalVolume(move!(solidworld), det.fWrldMaterial, "World")
    physWorld  = G4PVPlacement(nothing,          # no rotation
                              G4ThreeVector(),  # at (0,0,0)
                              logicworld,       # its logical volume
                              "World",          # its name
                              nothing,          # its mother volume
                              false,            # no boolean operation
                              0,                # copy number
                              checkOverlaps)                # overlaps checking 
    # Crystal
    solidcrst = G4Box("CRYSTAL", 0.5 * crstXY, 0.5 * crstXY, 0.5 * crstZ)
    logiccrst = G4LogicalVolume(CxxPtr(solidcrst), det.fCrstMaterial, "CRYSTAL")
  
    G4PVPlacement(nothing,           # no rotation
                  G4ThreeVector(),   # at (0,0,0)
                  logiccrst,          # its logical volume
                  "CRYSTAL",            # its name
                  logicworld,        # its mother  volume
                  false,             # no boolean operation
                  0,                 # copy number
                  checkOverlaps)     # overlaps checking
    
    SetVisAttributes(logicworld, G4VisAttributes!GetInvisible())
    SetVisAttributes(logiccrst, G4VisAttributes(G4Colour(1.0, 1.0, 0.0)))
    
    return physWorld              # return a pointer to the G4PhysicalVolume
end

crstDetectorConstruction (generic function with 1 method)

In [3]:
function lysoDetectorConstruction(det::CrstDetector)::CxxPtr{G4VPhysicalVolume}
    (; checkOverlaps, crstXY, crstZ)  = det

    crystBox = G4Box("CBox", crstXY/2,crstXY/2,crstZ/2)
    #nist     = G4NistManager!Instance()
    #cm3      = cm * cm * cm
    #lyso = G4Material("lyso", density= 7.4*g/cm3, ncomponents=4)
    
    #Lu = FindOrBuildElement(nist,"Lu")
    #Si = FindOrBuildElement(nist,"Si")
    #O  = FindOrBuildElement(nist,"O")
    #Y  = FindOrBuildElement(nist,"Y")
    #AddElement(lyso, Lu, fractionmass=.71)
    #AddElement(lyso, Si, fractionmass=.07)
    #AddElement(lyso, O, fractionmass=.18)
    #AddElement(lyso, Y, fractionmass=.04)
  
    #air  = FindOrBuildMaterial(nist, "G4_AIR")

    world_sizeXY = 2* crstXY
    world_sizeZ  = 2* crstZ
    
    solidworld = G4Box("World",  0.5*world_sizeXY, 0.5*world_sizeXY, 0.5*world_sizeZ)   
    logicworld = G4LogicalVolume(move!(solidworld), det.fWrldMaterial, "World")
    physWorld  = G4PVPlacement(nothing,          # no rotation
                              G4ThreeVector(),  # at (0,0,0)
                              logicworld,       # its logical volume
                              "World",          # its name
                              nothing,          # its mother volume
                              false,            # no boolean operation
                              0,                # copy number
                              checkOverlaps)                # overlaps checking 

        
    # Crystal
    solidcrst = G4Box("LYSO", 0.5 * crstXY, 0.5 * crstXY, 0.5 * crstZ)
    logiccrst = G4LogicalVolume(CxxPtr(solidcrst), det.fCrstMaterial, "LYSO")
  
    G4PVPlacement(nothing,           # no rotation
                  G4ThreeVector(),   # at (0,0,0)
                  logiccrst,          # its logical volume
                  "LYSO",            # its name
                  logicworld,        # its mother  volume
                  false,             # no boolean operation
                  0,                 # copy number
                  checkOverlaps)     # overlaps checking

           

    #SDz = 0.15*mm
    #gap = 0.01*mm;
  
    #solidDetector = G4Box("SiPM", 0.5 * crstXY, 0.5 * crstXY, SDz)

    #logicDetector = G4LogicalVolume(move!(solidDetector), air, "SiPM")

    #G4PVPlacement(nothing,           # no rotation
    #              G4ThreeVector(0.,0.,0.5*crstZ + SDz + gap),
     #             logicDetector,
     #             "SiPMR",
     #             logicworld,        # its mother  volume
      #            false,             # no boolean operation
     #             0,                 # copy number
     #             checkOverlaps)     # overlaps checking
  #  G4PVPlacement(nothing,           # no rotation
   #               G4ThreeVector(0.,0.,-0.5*crstZ - SDz - gap),
   #               logicDetector,
    #              "SiPML",
    #              logicworld,        # its mother  volume
    #              false,             # no boolean operation
     #             0,                 # copy number
     #             checkOverlaps)     # overlaps checking


  # Visualization attributes              
    SetVisAttributes(logicworld, G4VisAttributes!GetInvisible())
    SetVisAttributes(logiccrst, G4VisAttributes(G4Colour(1.0, 1.0, 0.0)))
    #SetVisAttributes(logicDetector,  G4VisAttributes(G4Colour(0.0, 0.0, 1.0)))
                  
    return physWorld              # return a pointer to the G4PhysicalVolume
end




lysoDetectorConstruction (generic function with 1 method)

In [4]:
Geant4.getConstructor(::CrstDetector)::Function = crstDetectorConstruction

In [5]:
particlegun = G4JLGunGenerator(particle = "gamma", 
                               energy = 511keV, 
                               direction = G4ThreeVector(0,0,1), 
                               position = G4ThreeVector(0,0,0))

G4JLGunGenerator("ParticleGun", Geant4.G4JLParticleGunData(nothing, "gamma", G4ThreeVector(0.0,0.0,1.0), G4ThreeVector(0.0,0.0,0.0), 0.511), Geant4.var"#init#23"(), Geant4.var"#gen#24"(), G4JLGeneratorAction[])

In [6]:
app = G4JLApplication( detector = CrstDetector(checkOverlaps=true,
                                   crstXY=3.5mm, crstZ=10.0mm, crstName="BGO"), # detector with parameters
                       generator = particlegun,                      # primary particle generator
                       nthreads = 1,                                 # number of threads (MT)
                       physics_type = QBBC,                     # what physics list to instantiate
                      )


**************************************************************
 Geant4 version Name: geant4-11-01-patch-01 [MT]   (10-February-2023)
  << in Multi-threaded mode >> 
                       Copyright : Geant4 Collaboration
                      References : NIM A 506 (2003), 250-303
                                 : IEEE-TNS 53 (2006), 270-278
                                 : NIM A 835 (2016), 186-225
                             WWW : http://geant4.org/
**************************************************************



G4JLApplication{CrstDetector, Geant4.G4JLNoData}(Geant4.G4MTRunManagerAllocated(Ptr{Nothing} @0x000000012ec2a200), CrstDetector(true, 3.5, 10.0, "BGO", CxxPtr{G4Material}(Ptr{G4Material} @0x000000013e60fe50), CxxPtr{G4Material}(Ptr{G4Material} @0x000000013e60e470)), [Geant4.G4JLNoData(), Geant4.G4JLNoData()], G4JLGunGenerator("ParticleGun", Geant4.G4JLParticleGunData(nothing, "gamma", G4ThreeVector(0.0,0.0,1.0), G4ThreeVector(0.0,0.0,0.0), 0.511), Geant4.var"#init#23"(), Geant4.var"#gen#24"(), G4JLGeneratorAction[]), nothing, 1, 0, G4MTRunManager, G4JLDetectorConstruction, QBBC, G4JLRunAction, G4JLEventAction, G4JLTrackingAction, G4JLSteppingAction, nothing, nothing, nothing, nothing, nothing, nothing, nothing, Dict{String, Geant4.G4JLProtoSD}(), Dict{String, Vector{G4JLSensitiveDetector}}(), G4JLScoringMesh[], nothing, nothing)

In [7]:
configure(app)

In [8]:
initialize(app)

Checking overlaps for volume CRYSTAL:0 (G4Box) ... OK! 


In [9]:
beamOn(app, 1)

In [10]:
world = GetWorldVolume()

Geant4.G4VPhysicalVolumeDereferenced(Ptr{Nothing} @0x0000600003854370)

In [ ]:
fig = Figure(resolution=(1024,1024))
s = LScene(fig[1,1])
Geant4.draw!(s, world)
#for t in tracks
#    lines!(t)
#end
display(fig)